#### Deep Learning Assignment 2 (conda env:tensorflow) fullyconnected

Previously in 1_notmnist.ipynb, we created a pickle with formatted datasets for training, development and testing on the notMNIST dataset.

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.


In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in 1_notmnist.ipynb.

In [3]:
### from  1_notmnist.ipynb
# pickle_file = 'notMNIST.pickle'

# try:
#   f = open(pickle_file, 'wb')
#   save = {
#     'train_dataset': train_dataset,
#     'train_labels': train_labels,
#     'valid_dataset': valid_dataset,
#     'valid_labels': valid_labels,
#     'test_dataset': test_dataset,
#     'test_labels': test_labels,
#     }
#   pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
#   f.close()

In [4]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


#### Reformat into a shape that's more adapted to the models we're going to train:

    data as a flat matrix,
    labels as float 1-hot encodings.



In [5]:
#test
print (28*28)
a1= np.arange(0,12).reshape(2,6)
a2=np.arange(0,24)
a3 = a2.reshape(3, 2 ,4)
print (a1)
print ("**a2 : ",a2)
print ("**a3 :\n ",a3)
print("*"*50)
i_size = 2
a4=a2.reshape(6,i_size,i_size)
print ("*** a4 : \n",a4)
a5=a4.reshape(-1, i_size * i_size)
print("****a4.reshape(-1, i_size * i_size) : reshape(-1,4) \n",a5)
print ("-"*70)
a6 = np.arange(3 * 24).reshape(3, 6, 4)
print ("*** a6 = reshape np.arange(3 * 24).reshape(3, 6, 4):\n",a6)
print ("***  a6.reshape(3 , 24):\n",a6.reshape(3 , 24))
print ("***  a6.reshape(3 , -1):\n",a6.reshape(3 , -1))  # -i column :rest number of 24row which makes same 24by3 array
print ("***  a6.reshape(-1 , 24):\n",a6.reshape(-1 , 24))

784
[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]]
**a2 :  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
**a3 :
  [[[ 0  1  2  3]
  [ 4  5  6  7]]

 [[ 8  9 10 11]
  [12 13 14 15]]

 [[16 17 18 19]
  [20 21 22 23]]]
**************************************************
*** a4 : 
 [[[ 0  1]
  [ 2  3]]

 [[ 4  5]
  [ 6  7]]

 [[ 8  9]
  [10 11]]

 [[12 13]
  [14 15]]

 [[16 17]
  [18 19]]

 [[20 21]
  [22 23]]]
****a4.reshape(-1, i_size * i_size) : reshape(-1,4) 
 [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]
 [16 17 18 19]
 [20 21 22 23]]
----------------------------------------------------------------------
*** a6 = reshape np.arange(3 * 24).reshape(3, 6, 4):
 [[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]
  [12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]

 [[24 25 26 27]
  [28 29 30 31]
  [32 33 34 35]
  [36 37 38 39]
  [40 41 42 43]
  [44 45 46 47]]

 [[48 49 50 51]
  [52 53 54 55]
  [56 57 58 59]
  [60 61 62 63]
  [64 65 66 67]
  [68 69 70 71]]]
**

In [6]:
#test
image_size = 28
print (train_dataset.shape) #3d array
print (type(train_dataset))
dataset = train_dataset.reshape(-1, image_size * image_size)  # -1 represnests x: x*(28*28) = 200000*28*28
print (dataset.shape)       
print (dataset)

(200000, 28, 28)
<type 'numpy.ndarray'>
(200000, 784)
[[-0.46470588 -0.29215688 -0.11176471 ..., -0.5        -0.5        -0.5       ]
 [ 0.5         0.5         0.5        ..., -0.5        -0.49215686
  -0.49607843]
 [-0.5        -0.5        -0.5        ..., -0.5        -0.5        -0.5       ]
 ..., 
 [ 0.41764706  0.5         0.49607843 ..., -0.49215686 -0.5        -0.5       ]
 [-0.5        -0.5        -0.5        ..., -0.5        -0.5        -0.5       ]
 [ 0.5         0.5         0.5        ...,  0.49215686  0.07647059
  -0.28039217]]


In [45]:
#test
num_labels = 10
print ("** np.arange(num_labels):\n",np.arange(num_labels))
print ("** train_labels.shape: ",train_labels.shape)  #2d
print ("train_labels :\n",train_labels)
print ("** train_labels[:,None].shape:",train_labels[:,None].shape)  #convert to 3d array(200000, 1, 10) from 2d 
print ("train_labels[:,None]: \n",train_labels[:,None])
test_a = np.arange(24).reshape(4,6)
print ("***** test_a: \n",test_a)
print(test_a.shape)
print ("***** test_a[:,None] :\n",test_a[:,None])  #4 row of (1by6 array) array : convert to 3d array from 2d array
print (test_a[:,None].shape)

print ("*"*90)
labels2 = (np.arange(num_labels) == train_labels[:,None]).astype(np.float32)
print ("1-hot encoding:\n",labels2) #3d (200000, 1, 10) with 1-hot encoding
print (labels2.shape)

** np.arange(num_labels):
 [0 1 2 3 4 5 6 7 8 9]
** train_labels.shape:  (200000, 10)
train_labels :
 [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
** train_labels[:,None].shape: (200000, 1, 10)
train_labels[:,None]: 
 [[[ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  0.]]

 ..., 
 [[ 0.  1.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  1.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  0.]]]
***** test_a: 
 [[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]]
(4, 6)
***** test_a[:,None] :
 [[[ 0  1  2  3  4  5]]

 [[ 6  7  8  9 10 11]]

 [[12 13 14 15 16 17]]

 [[18 19 20 21 22 23]]]
(4, 1, 6)
******************************************************************************************
1-hot encoding:
 [[[ 1.  0.  0. ...,  0.  0.  0.]]

 [[ 1.  0.  0. ...,  0.  0.  

In [8]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)  #convert to 2d datasetn fm 3d
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)  #1-hoot encoding
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


#### We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:

    First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

    with graph.as_default():
        ...

    Then you can run the operations on this graph as many times as you want by calling session.run(), providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

    with tf.Session(graph=graph) as session:
        ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:
 

In [9]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

#### Let's run this computation and iterate:

In [29]:
num_steps = 801

def accuracy(predictions, labels):
#predictions.shape : (10000, 10), so np.argmax(predictions, 1) is the max index among 10 columns
#np.argmax(labels,1) : (10000, 10),  the max index among 10 columns
## for each 10000 row ,100*max index among 10 predictions column == max index among 10 labels column.
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0]) #10000(row):total number of predcition

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):   #9 occurrence(0,100,..800)
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
#test_prediction:   ,test_labels:(10000, 10)2d-array
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))  

Initialized
Loss at step 0: 17.664175
Training accuracy: 7.3%
Validation accuracy: 10.6%
Loss at step 100: 2.338831
Training accuracy: 71.5%
Validation accuracy: 70.7%
Loss at step 200: 1.872660
Training accuracy: 75.0%
Validation accuracy: 73.1%
Loss at step 300: 1.622114
Training accuracy: 76.0%
Validation accuracy: 73.8%
Loss at step 400: 1.454489
Training accuracy: 76.8%
Validation accuracy: 74.3%
Loss at step 500: 1.330898
Training accuracy: 77.7%
Validation accuracy: 74.5%
Loss at step 600: 1.234529
Training accuracy: 78.3%
Validation accuracy: 74.7%
Loss at step 700: 1.156531
Training accuracy: 78.9%
Validation accuracy: 74.7%
Loss at step 800: 1.091674
Training accuracy: 79.1%
Validation accuracy: 74.7%
Test accuracy: 83.2%


In [37]:
#test
print (test_labels.shape)
print (predictions.shape)
print (test_labels.shape)

print("** predictions :\n",predictions)
print("*"*50)
print("** test_labels :\n",test_labels)
#above cell is not excuted , predictions not ref, so maybe error 
print((100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(test_labels, 1))
          / predictions.shape[0]))

(10000, 10)
(10000, 10)
(10000, 10)
** predictions :
 [[  1.21644632e-07   4.27734676e-06   2.16982838e-11 ...,   2.83619208e-08
    5.43003997e-09   1.97109884e-09]
 [  2.35422485e-05   1.23736542e-02   2.87960520e-05 ...,   8.65043967e-06
    2.45758406e-06   7.28015766e-06]
 [  3.48756475e-06   1.15768451e-07   3.30291072e-10 ...,   9.48472746e-07
    4.87349041e-07   2.79515300e-09]
 ..., 
 [  5.55673055e-03   2.63089269e-01   2.56393407e-03 ...,   1.82805394e-07
    1.03649870e-02   1.81789419e-05]
 [  2.81543677e-08   3.16697668e-09   2.29063705e-02 ...,   6.74104956e-12
    4.46748231e-07   6.30877594e-07]
 [  1.68385927e-09   1.11338549e-09   3.33384742e-09 ...,   1.06131068e-08
    1.16494618e-08   9.99999523e-01]]
**************************************************
** test_labels :
 [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 ..., 
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.

#### Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a <span style="color:red">Placeholder node </span>which will be fed actual data at every call of session.run().


In [61]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)


####  Let's run it:

In [62]:


num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))



Initialized
Minibatch loss at step 0: 18.464163
Minibatch accuracy: 11.7%
Validation accuracy: 18.0%
Minibatch loss at step 500: 1.672913
Minibatch accuracy: 78.9%
Validation accuracy: 75.6%
Minibatch loss at step 1000: 1.881558
Minibatch accuracy: 71.1%
Validation accuracy: 76.6%
Minibatch loss at step 1500: 1.121875
Minibatch accuracy: 76.6%
Validation accuracy: 76.7%
Minibatch loss at step 2000: 0.743603
Minibatch accuracy: 77.3%
Validation accuracy: 77.6%
Minibatch loss at step 2500: 0.808663
Minibatch accuracy: 82.8%
Validation accuracy: 78.0%
Minibatch loss at step 3000: 1.262544
Minibatch accuracy: 74.2%
Validation accuracy: 77.7%
Test accuracy: 85.5%


####  Problem

Turn the logistic regression example with SGD into a <span style="color:red">1-hidden layer neural network </span>with rectified linear units <span style="color:red">nn.relu() </span>and 1024 Turn the logistic regression example with SGD into a <span style="color:red">1-hidden layer neural network </span>with rectified linear units <span style="color:red">nn.relu() </span>and 1024 <span style="color:red">hidden nodes</span>. This model should improve your validation / test accuracy.
hidden nodes. This model should improve your validation / test accuracy.


In [64]:
#ver1
hidden_nodes = 1024
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Hidden Layer
  weights_hidden = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_hidden = tf.Variable(tf.zeros([hidden_nodes]))

  # RELU
  relu = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)

  # Output layer
  weights_output = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases_output = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.matmul(relu, weights_output) + biases_output
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  print ("train_prediction size = ", train_prediction.get_shape())

  valid_prediction = tf.nn.softmax(tf.matmul(
                                    tf.nn.relu(
                                      tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden), 
                                    weights_output) + biases_output)
  print ("valid_prediction size = ", valid_prediction.get_shape())
  test_prediction = tf.nn.softmax(tf.matmul(
                                    tf.nn.relu(
                                      tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden),
                                    weights_output) + biases_output)
  print ("test_prediction size = ", test_prediction.get_shape())

train_prediction size =  (128, 10)
valid_prediction size =  (10000, 10)
test_prediction size =  (10000, 10)


In [66]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print ("Initialized")
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print ("Minibatch loss at step", step, ":", l)
      print ("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print ("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print ("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0 : 297.042
Minibatch accuracy: 18.0%
Validation accuracy: 34.1%
Minibatch loss at step 500 : 13.8108
Minibatch accuracy: 78.1%
Validation accuracy: 77.8%
Minibatch loss at step 1000 : 20.8685
Minibatch accuracy: 76.6%
Validation accuracy: 77.2%
Minibatch loss at step 1500 : 2.30572
Minibatch accuracy: 90.6%
Validation accuracy: 81.7%
Minibatch loss at step 2000 : 3.76205
Minibatch accuracy: 82.0%
Validation accuracy: 82.2%
Minibatch loss at step 2500 : 5.25211
Minibatch accuracy: 84.4%
Validation accuracy: 80.8%
Minibatch loss at step 3000 : 3.04822
Minibatch accuracy: 75.0%
Validation accuracy: 82.5%
Test accuracy: 90.0%
